In [1]:
# This program merges OEC (with opportunity variables) and GTA dataset

# TODO

- Investigate left-only merge

# Loading packages

In [2]:
import numpy as np
import pandas as pd
import os
from google.colab import drive

# Setting the ambience

In [3]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
work_dir                            = '/content/gdrive/My Drive/ip_complexity/create_finaldataset'
output_dir                          = '/content/gdrive/My Drive/ip_complexity/create_finaldataset/output'
create_dataset_gta_dir              = '/content/gdrive/My Drive/ip_complexity/create_dataset_gta/output/data'
create_variables_opportunity_dir    = '/content/gdrive/My Drive/ip_complexity/oec_data/create_variables_opportunity/output/data'

# Oppening OEC dataset

In [5]:
df_oec = pd.read_csv(create_variables_opportunity_dir + "/dataset_oec_with_opportunity.csv")

In [6]:
# Convert 'hs4_id' column to string and fill with zeros until it reachs 4 digits
df_oec["hs4_id"] = df_oec["hs4_id"].astype(str).str.zfill(4)

In [7]:
df_oec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3882912 entries, 0 to 3882911
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   country_id           object 
 2   country              object 
 3   country_eci          float64
 4   hs4_id               object 
 5   hs4_name             object 
 6   hs4_pci              float64
 7   tv_relatedness       float64
 8   tv_rca               float64
 9   opportunity_index_1  float64
 10  opportunity_index_2  float64
 11  opportunity_index_3  float64
 12  tv_rca_bigger_1      int64  
dtypes: float64(7), int64(2), object(4)
memory usage: 385.1+ MB


In [8]:
df_oec = df_oec.sort_values(by = 'hs4_id')

# Oppening GTA dataset

In [9]:
df_gta = pd.read_csv(create_dataset_gta_dir + "/dataset_gta_intervention_product.csv", sep = ";")

<ipython-input-9-ee6438536bb8>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gta = pd.read_csv(create_dataset_gta_dir + "/dataset_gta_intervention_product.csv", sep = ";")


In [10]:
# Define the renaming function
rename_func = lambda x: x.lower().replace(' ', '_')
# Rename the columns using the function
df_gta = df_gta.rename(columns=rename_func)

In [11]:
df_gta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574565 entries, 0 to 6574564
Data columns (total 11 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   state_act_id               int64 
 1   intervention_id            int64 
 2   state_act_title            object
 3   announcement_date          object
 4   gta_evaluation             object
 5   currently_in_force         object
 6   implementing_jurisdiction  object
 7   intervention_type          object
 8   mast_chapter               object
 9   affected_sectors           object
 10  affected_product           object
dtypes: int64(2), object(9)
memory usage: 551.8+ MB


In [12]:
df_gta = df_gta.rename(columns = {"implementing_jurisdiction": "country",
                         "affected_product": "hs4_id"
                        })

In [13]:
# Convert "announcement_data" column to date type
df_gta["announcement_date"] = pd.to_datetime(df_gta["announcement_date"])

# Create a variable "year" with the year from "announcement_data"
df_gta["year"] = df_gta["announcement_date"].dt.year

In [14]:
# Extract the first four digits from 'hs4_id' to merge with OEC data
df_gta["hs4_id"] = df_gta["hs4_id"].astype(str).str[:4]

In [15]:
# Convert 'hs4_id' column to string and fill with zeros until it reachs 4 digits
df_gta["hs4_id"] = df_gta["hs4_id"].astype(str).str.zfill(4)

In [16]:
df_gta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574565 entries, 0 to 6574564
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   state_act_id        int64         
 1   intervention_id     int64         
 2   state_act_title     object        
 3   announcement_date   datetime64[ns]
 4   gta_evaluation      object        
 5   currently_in_force  object        
 6   country             object        
 7   intervention_type   object        
 8   mast_chapter        object        
 9   affected_sectors    object        
 10  hs4_id              object        
 11  year                int64         
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 601.9+ MB


In [17]:
# drop duplicates
df_gta = df_gta.drop_duplicates()

In [18]:
df_gta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3469498 entries, 0 to 6574564
Data columns (total 12 columns):
 #   Column              Dtype         
---  ------              -----         
 0   state_act_id        int64         
 1   intervention_id     int64         
 2   state_act_title     object        
 3   announcement_date   datetime64[ns]
 4   gta_evaluation      object        
 5   currently_in_force  object        
 6   country             object        
 7   intervention_type   object        
 8   mast_chapter        object        
 9   affected_sectors    object        
 10  hs4_id              object        
 11  year                int64         
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 344.1+ MB


# Merging 

In [19]:
df = pd.merge(df_gta, df_oec, on=['hs4_id', 'year', 'country'], how = "outer", indicator = True)

In [20]:
df["_merge"].value_counts() # right_only means that a product p was not protected/liberalized by country c at year j. It is ok to drop, since we're interested only in the products that are target by policies


right_only    3608565
both          2426825
left_only     1042673
Name: _merge, dtype: int64

In [33]:
df[df["_merge"] == "left_only"].year.value_counts()

2022    300281
2015    147825
2020     92025
2014     79701
2021     60379
2013     50221
2018     49655
2019     41702
2010     40614
2012     40484
2011     38559
2023     26881
2016     25188
2009     24619
2017     20252
2008      4287
Name: year, dtype: int64

In [34]:
df[df["_merge"] == "left_only"].country.value_counts()

United States of America    237061
France                       84611
Germany                      63064
Spain                        61911
Italy                        49530
                             ...  
Bahamas                          1
Turkmenistan                     1
Guam                             1
New Caledonia                    1
Faeroe Islands                   1
Name: country, Length: 196, dtype: int64

In [21]:
df = df[df["_merge"] != "right_only"]

In [22]:
#df = df.drop(columns = '_merge')

In [23]:
# Drop the DataFrame from memory
del df_gta
del df_oec

In [24]:
# create protectionist/liberal dummies
# Create a dummy variable based on the condition 'color == "Red"'
df['protectionist'] = df['gta_evaluation'].apply(lambda x: 1 if x == 'Red' else (0 if x == 'Green' else np.nan))
df['liberal'] = df['gta_evaluation'].apply(lambda x: 1 if x == 'Green' else (0 if x == 'Red' else np.nan))

In [26]:
df.describe()

,state_act_id,intervention_id,year,country_eci,hs4_pci,tv_relatedness,tv_rca,opportunity_index_1,opportunity_index_2,opportunity_index_3,tv_rca_bigger_1,protectionist,liberal
count,3.469498e+06,3.469498e+06,3.469498e+06,2.425985e+06,2.413428e+06,2.426825e+06,2.426825e+06,2.413428e+06,2.413428e+06,2.413428e+06,2.426825e+06,3.338299e+06,3.338299e+06
mean,3.952206e+04,7.215623e+04,2.016323e+03,8.502530e-01,3.417798e-01,3.397625e-01,1.360237e+00,1.176235e-01,3.177675e-02,5.815142e-02,3.851444e-01,7.377727e-01,2.622273e-01
std,2.178352e+04,3.248483e+04,3.905801e+00,5.384924e-01,9.667030e-01,1.375071e-01,3.426373e+00,3.601785e-01,4.330025e+00,1.355858e+00,4.866295e-01,4.398456e-01,4.398456e-01
min,5.300000e+01,5.688000e+03,2.007000e+03,-2.343338e+00,-3.558203e+00,1.312343e-03,0.000000e+00,-1.860406e+00,-4.911820e+02,-1.463796e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.651100e+04,5.872000e+04,2.014000e+03,4.840182e-01,-3.286398e-01,2.411988e-01,2.576924e-01,-9.031292e-02,-8.652249e-02,-2.299402e-02,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.437000e+04,8.101900e+04,2.016000e+03,8.617006e-01,5.953570e-01,3.578778e-01,6.899295e-01,1.378532e-01,1.759326e-01,4.076422e-02,0.000000e+00,1.000000e+00,0.000000e+00
75%,5.490400e+04,9.307800e+04,2.020000e+03,1.271406e+00,1.064076e+00,4.425380e-01,1.501526e+00,3.491689e-01,8.036211e-01,2.901872e-01,1.000000e+00,1.000000e+00,1.000000e+00
max,7.337800e+04,1.188140e+05,2.023000e+03,2.260692e+00,3.153356e+00,9.479168e-01,2.531716e+02,1.736715e+00,1.162207e+02,4.968227e+01,1.000000e+00,1.000000e+00,1.000000e+00


# Saving

In [27]:
# Specify the desired column order
column_order = ['year', 'country', 'country_id', 'country_eci', 'state_act_id', 'intervention_id', 'protectionist', 'liberal']  # Add more variables as needed

# Reorder the DataFrame columns
df = df[column_order + list(df.columns.difference(column_order))]

In [28]:
df.to_csv(output_dir + '/data/oec_gta_dataset.csv', index = False)